In [1]:
from __future__ import print_function
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

#读取数据
digits = load_digits()
X = digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.3)

#构建神经网络函数
def add_layer(inputs,in_size,out_size,layer_name,activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size,out_size]))
    biases = tf.Variable(tf.zeros([1,out_size])+0.1)
    Wx_plus_b = tf.add(tf.matmul(inputs,Weights),biases)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
        tf.summary.histogram(layer_name+'/outputs',outputs)
        return outputs

#设置placeholder对神经网络进行数据的传入传出
xs = tf.placeholder(tf.float32,[None,64])#这是一个8*8的数据
ys = tf.placeholder(tf.float32,[None,10])#这里的10 是分别代表0-9的

#添加神经网络层，第一个是隐藏层，第二个是输出层
l1 = add_layer(xs,64,100,'l1',activation_function=tf.nn.tanh)

prediction = add_layer(l1,100,10,'l2',activation_function=tf.nn.softmax)


#构建代价函数 ,
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),reduction_indices=[1]))
tf.summary.scalar('loss',cross_entropy)  #只要是查看误差的这一步中，使用的都是scalar，因为得到的误差是一个标量，不是一个矢量
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

#只要涉及到变量的，都必须会使用到这一最重要的步骤
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)


merged = tf.summary.merge_all() #这一步就是把所有要查看的过程，都融合起来
#记录summary 要记录两个，一个是train_data的，一个是test_data的summary
train_writer = tf.summary.FileWriter('log/train',sess.graph) 
test_writer = tf.summary.FileWriter('log/test',sess.graph)


#开始训练
for i in range(500):
    sess.run(train_step,feed_dict={xs:X_train,ys:y_train})
    if i % 50 == 0:
        train_result = sess.run(merged,feed_dict={xs:X_train,ys:y_train})
        test_result = sess.run(merged,feed_dict={xs:X_test,ys:y_test})
        
        train_writer.add_summary(train_result,i)
        test_writer.add_summary(test_result,i)
        
        
####若要运行这个程序，需要解决一下问题
#解决问题的  网址在这  https://github.com/h5py/h5py/issues/961
#需要运行这个指令  conda install h5py==2.8.0
        